# Report 2

## Task 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.animation as animation
import os

#You can change the file that you are trying to read here:

filename = "walking_1"
folder = "hw2data/train/"

vals = np.load(folder + filename + ".npy")

print(vals.shape)

xyz = np.reshape( vals[:, :], [38, 3, -1])

print(xyz.shape)

(114, 100)
(38, 3, 100)


In [2]:
r = 1000
xroot, yroot, zroot = xyz[0,0,0], xyz[0, 0, 1], xyz[0,0, 2]

I = np.array(
        [1, 2, 3, 4, 5, 6, 1, 8, 9, 10, 11, 12, 1, 14, 15, 16, 17, 18, 19, 16, 21, 22, 23, 25, 26, 24, 28, 16, 30, 31,
         32, 33, 34, 35, 36, 37]) - 1
J = np.array(
        [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32,
         33, 34, 35, 36, 37, 38]) - 1

for joint in range(1, xyz.shape[2]):

    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    for move in range(0, I.shape[0]):
        xline = np.array(xyz[I[move], 0, joint], xyz[J[move], 0, joint])
        yline = np.array(xyz[I[move], 1, joint], xyz[J[move], 1, joint])
        zline = np.array(xyz[I[move], 2, joint], xyz[J[move], 2, joint])
        ax.plot(xline, yline, zline)
    
    ax.set_xlim([-r + xroot, r + xroot])
    ax.set_ylim([-r + yroot, r + yroot])
    ax.set_zlim([-r + zroot, r + zroot])

    plt.savefig(f"{joint}.png")
    plt.close()

from PIL import Image

images = [Image.open(f"{n}.png") for n in range(1, xyz.shape[2])]
images[0].save(filename + '.gif', save_all=True, append_images=images[1:], duration=30, loop=0)

for n in range(1, xyz.shape[2]):
    os.remove(f"{n}.png")